In [18]:
%%javascript

IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [19]:
import pandas as pd
import numpy as np
import re
import igraph as ig
from os import listdir

from functions import *
from groupClasses import *

In [27]:
class SymmetryAnalyzer:
	def __init__(self, fileName):
		self.parseInputFile(fileName)
		
		self.ig_graph = ig.Graph.TupleList(self.edges[["Source", "Target", "Color"]].itertuples(index = False),
										   directed = False, edge_attrs = "color")
		self.pynauty_graph = get_pynauty_graph(self.ig_graph)
		
		self.generators, grpsize1, grpsize2, orbits, numorbits = pynauty.autgrp(self.pynauty_graph)
		self.permutations = [combinatorics.Permutation(generator) for generator in self.generators]
	
	def decompose(self):
		self.sectors = get_sectors(self.permutations)
		
		self.NormalSubgroups = [NormalSubgroup(self.ig_graph, self.sectors[i],
											   get_permutations_on_sector_id(self.permutations,
																			 self.sectors, i))
								for i in range(len(self.sectors))]
		
		self.calculate_epsilons()
	
	def calculate_indices(self):
		self.calculate_fiedler_values()
		self.calculate_experimental_values()
	
	#############################################################################
	########################## Calculating indices ##############################
	#############################################################################
	def calculate_epsilons(self):
		for NormalSubgroup in self.NormalSubgroups:
			NormalSubgroup.calculate_epsilons()
		
		meanEpsilons = [NormalSubgroup.meanEpsilon for NormalSubgroup in self.NormalSubgroups]
		self.meanEpsilon = mean(meanEpsilons)
	
	def calculate_fiedler_values(self):
		graph_laplacian = self.ig_graph.laplacian(normalized = False)
		graph_laplacian = np.array(graph_laplacian)
		eigenval, eigenvect = np.linalg.eig(graph_laplacian)
		# we only take the real part, because imaginary part is very small and
		# it seems like it's here due to a computational artifact
		eigenval = eigenval.real
		eigenval = np.sort(eigenval)
		self.fiedlerValue = eigenval[1]
		self.eigenRatio = eigenval[1] / eigenval[len(eigenval) - 1]
		
		graph_laplacian = self.ig_graph.laplacian(normalized = True)
		graph_laplacian = np.array(graph_laplacian)
		eigenval, eigenvect = np.linalg.eig(graph_laplacian)
		# we only take the real part, because imaginary part is very small and
		# it seems like it's here due to a computational artifact
		eigenval = eigenval.real
		eigenval = np.sort(eigenval)
		self.normalizedFiedlerValue = eigenval[1]
	
	def calculate_experimental_values(self):
		adjacency_matrix = np.array(self.ig_graph.get_adjacency().data)
		eigenval, eigenvect = np.linalg.eig(adjacency_matrix)
		eigenval = np.sort(eigenval)
		kappa = eigenval[len(eigenval) - 1] + 5
		
		graph_laplacian = self.ig_graph.laplacian(normalized = False)
		graph_laplacian = np.array(graph_laplacian)
		degreeSeq = np.array(self.ig_graph.degree())
		randomWalkLaplacian = graph_laplacian / degreeSeq[:, None]
		eigenval, eigenvect = np.linalg.eig(randomWalkLaplacian)
		# we only take the real part, because imaginary part is very small and
		# it seems like it's here due to a computational artifact
		eigenval = eigenval.real
		eigenval = np.sort(eigenval)
		self.randomWalkSecondEigenvalue = eigenval[1]
		
		degreeSeq = np.sort(np.array(self.ig_graph.degree()))
		self.minimalDegree = np.ndarray.min(degreeSeq)
		self.maximalDegree = np.ndarray.max(degreeSeq)
		self.sumDegrees = np.sum(degreeSeq)
		
		self.vertex_connectivity = self.ig_graph.vertex_connectivity()
		self.edge_connectivity = self.ig_graph.edge_connectivity()
	#############################################################################
	############################## Input/output #################################
	#############################################################################
	################# Input #################
	def parseInputFile(self, inputFile):
		inputData = pd.read_table(inputFile, sep = "\n", header = None, names = ["Data"])
		
		nodes = inputData.loc[(np.flatnonzero(inputData['Data'] == "Node,Color")[0] + 1):
							  (np.flatnonzero(inputData['Data'] == "Edges added")[0] - 1)]
		nodes = nodes['Data'].str.split(",", expand = True)
		self.nodes = nodes.rename(columns={0: "Name", 1: "Color"})
		
		originalEdges = inputData.loc[(np.flatnonzero(inputData['Data'] == "Input File")[0] + 2):
									  (np.flatnonzero(inputData['Data'] == "Fixed File")[0] - 1)]
		originalEdges = originalEdges['Data'].str.split(" ", expand = True)
		originalEdges = originalEdges.rename(columns={0: "Source", 1: "Target", 2: "Weight"})
		originalEdges[["Source", "Target"]] = [sorted([row[0], row[1]]) for row in originalEdges.values]
		originalEdges = originalEdges.drop_duplicates()
		originalEdges["Color"] = "Original"
		
		repairedEdges = inputData.loc[(np.flatnonzero(inputData['Data'] == "Edges added")[0] + 1):
									  (np.flatnonzero(inputData['Data'] == "Input File")[0] - 1)]
		if not(repairedEdges.empty):
			repairedEdges = repairedEdges['Data'].str.split(",", expand = True)
			repairedEdges = repairedEdges.rename(columns={0: "Source", 1: "Target", 2: "Id", 3: "Weight"})
			repairedEdges = repairedEdges[["Source", "Target", "Weight"]]
			repairedEdges[["Source", "Target"]] = [sorted([row[0], row[1]]) for row in repairedEdges.values]
			repairedEdges = repairedEdges.drop_duplicates()
			repairedEdges["Color"] = "Repaired"
			self.edges = pd.concat([originalEdges, repairedEdges])
		else:
			self.edges = originalEdges
	
	################# Output #################
	def print_all_info(self):
		self.print_indices()
		print()
		self.print_normal_subgroups()
	
	def print_all_info_to_files(self, prefix):
		self.print_indices_to_file(prefix + "_indices.txt")
		self.print_normal_subgroups_to_file(prefix + "_normal_subgroups.txt")
		self.print_permutations_to_file(prefix + "_permutations.txt")
		self.print_edges_to_file(prefix + "_edges.txt")
	
	def print_normal_subgroups(self):
		for NormalSubgroup in self.NormalSubgroups:
			NormalSubgroup.print()
			print()
	
	def print_normal_subgroups_to_file(self, fileName):
		with open(fileName, 'w') as filehandle:
			pass
		for NormalSubgroup in self.NormalSubgroups:
			NormalSubgroup.print_info_to_file(fileName)

	def print_permutations_to_file(self, fileName):
		with open(fileName, 'w') as filehandle:
			filehandle.writelines("%s\n" % self.ig_graph.vs["name"])
		for NormalSubgroup in self.NormalSubgroups:
			NormalSubgroup.print_raw_permutations_to_file(fileName)
	
	def print_edges_to_file(self, fileName):
		edges = self.edges
		edges["Type"] = "Undirected"
		edges.to_csv(fileName, index = False)
	
	def print_indices(self):
		print("Fiedler value = ", self.fiedlerValue)
		print("Normalized Fiedler value = ", self.normalizedFiedlerValue)
		print("Eigen ratio = ", self.eigenRatio)
		print("Mean epsilon = ", self.meanEpsilon)
	
	def print_indices_to_file(self, fileName):
		with open(fileName, 'w') as filehandle:
			filehandle.writelines("Fiedler value,%s\n" % self.fiedlerValue)
			filehandle.writelines("Normalized Fiedler Value,%s\n" % self.normalizedFiedlerValue)
			filehandle.writelines("Eigen ratio,%s\n" % self.eigenRatio)
			filehandle.writelines("Mean epsilon,%s\n" % self.meanEpsilon)
			# experimental values from here down
			filehandle.writelines("Random Walk Fiedler Value,%s\n" % self.randomWalkSecondEigenvalue)
			filehandle.writelines("Minimal degree,%s\n" % self.minimalDegree)
			filehandle.writelines("Maximal degree,%s\n" % self.maximalDegree)
			filehandle.writelines("Sum of degrees,%s\n" % self.sumDegrees)
			filehandle.writelines("Vertex connectivity,%s\n" % self.vertex_connectivity)
			filehandle.writelines("Edge connectivity,%s\n" % self.edge_connectivity)

In [28]:
def runFile(inputFile, outputPrefix):
	print("Running ", inputFile)
	symmetryAnalyzer = SymmetryAnalyzer(inputFile)
	symmetryAnalyzer.decompose()
	symmetryAnalyzer.calculate_indices()
	symmetryAnalyzer.print_all_info_to_files(outputPrefix)

def runFolder(folderToRun, outputFolder):
	files = listdir(folderToRun)
	files = [file for file in files if re.search(r"\.[a-z]{3}$", file)]
	
	os.makedirs(outputFolder, exist_ok = True)
	for file in files:
		prefix = re.sub(r"\.[a-z]{3}$", "", file)
		runFile(folderToRun + "/" + file, outputFolder + "/" + prefix)

In [32]:
folderToRun = "../DATA/INTEGER-PROGRAM-COST-EDGES/INPUT/c_bw_no_05_12"
outputFolder = "../DATA/INTEGER-PROGRAM-COST-EDGES/OUTPUT/c_bw_no_05_12"
#folderToRun = "../DATA/INTEGER-PROGRAM-COST-EDGES/INPUT/c_fw_no_07_06"
#outputFolder = "../DATA/INTEGER-PROGRAM-COST-EDGES/OUTPUT/c_fw_no_07_06"
#folderToRun = "../DATA/HUBS-ONLY-NETWORKS/fw"
#outputFolder = "../DATA/HUBS-ONLY-NETWORKS/OUTPUT_FW"

runFolder(folderToRun, outputFolder)

Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_11.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_3.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_8.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_14.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_10.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_9.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_12.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_5.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_4.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_6.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_7.txt
Running  ../DATA/HUBS-ONLY-NETWORKS/fw/run_fw_hubs_only_nc_13.txt


In [17]:
symmetryAnalyzer = SymmetryAnalyzer("test.txt")
#symmetryAnalyzer = SymmetryAnalyzer("../DATA/INTEGER-PROGRAM-COST-EDGES/INPUT/c_fw_no_07_06/run_c_fw_no_nc_9.txt")
symmetryAnalyzer.decompose()
symmetryAnalyzer.calculate_indices()
symmetryAnalyzer.print_indices()

adjacency_matrix = np.array(symmetryAnalyzer.ig_graph.get_adjacency().data)
eigenval, eigenvect = np.linalg.eig(adjacency_matrix)
eigenval = np.sort(eigenval)
print(eigenval)
kappa = eigenval[len(eigenval) - 1] + 1
shifted_adjacency = adjacency_matrix - kappa * np.identity(len(adjacency_matrix))
eigenval, eigenvect = np.linalg.eig(shifted_adjacency)
eigenval = np.sort(eigenval)
eigenval

Fiedler value =  1.9999999999999998
Normalized Fiedler value =  1.0
Eigen ratio =  0.3999999999999998
Mean epsilon =  5.0
[-2.00000000e+00 -1.00000000e+00  0.00000000e+00  1.75044441e-16
  3.00000000e+00]


array([-6., -5., -4., -4., -1.])

In [11]:
import rpy2;
from rpy2 import robjects
robjects.r("version")

R object with classes: ('simple.list',) mapped to:
[StrSexpVe..., StrSexpVe..., StrSexpVe..., StrSexpVe..., ..., StrSexpVe..., StrSexpVe..., StrSexpVe..., StrSexpVe...]
  platform: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['x86_64-pc-linux-gnu']
  arch: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['x86_64']
  os: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['linux-gnu']
  system: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['x86_64, linux-gnu']
...
  major: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['77875']
  minor: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['R']
  year: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['R version 3.6.3 (2020-02-29)']
  month: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['Holding the Windsock']

In [9]:
# Using R inside python
import rpy2
import rpy2.robjects.packages as rpackages
base = rpackages.importr('base')
#base._libPaths()[int(0)]
base.version

R object with classes: ('simple.list',) mapped to:
[StrSexpVe..., StrSexpVe..., StrSexpVe..., StrSexpVe..., ..., StrSexpVe..., StrSexpVe..., StrSexpVe..., StrSexpVe...]
  platform: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['x86_64-pc-linux-gnu']
  arch: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['x86_64']
  os: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['linux-gnu']
  system: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['x86_64, linux-gnu']
...
  major: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['77875']
  minor: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['R']
  year: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['R version 3.6.3 (2020-02-29)']
  month: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['Holding the Windsock']

In [ ]:
# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind = int(1)) # select the first mirror in the list

# R package names
packnames = ('ggplot2', 'hexbin')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

R[write to console]: also installing the dependencies ‘colorspace’, ‘cli’, ‘crayon’, ‘utf8’, ‘farver’, ‘labeling’, ‘lifecycle’, ‘munsell’, ‘R6’, ‘RColorBrewer’, ‘viridisLite’, ‘ellipsis’, ‘fansi’, ‘magrittr’, ‘pillar’, ‘pkgconfig’, ‘vctrs’, ‘glue’, ‘gtable’, ‘isoband’, ‘rlang’, ‘scales’, ‘tibble’, ‘withr’


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/colorspace_2.0-2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2196563 bytes (2.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

R[write to console]: downloaded 10156 bytes


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/lifecycle_1.0.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 83389 bytes (81 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to cons

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

R[write to console]: downloaded 8066 bytes


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/fansi_0.5.0.tar.gz'

R[write to

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to 

R[write to console]: 

R[write to console]: downloaded 677 KB


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/withr_2.4.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 94283 bytes (92 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R

In [13]:
# Using R inside python
import rpy2
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects.packages import importr
#base = importr('base')
#base._libPaths()[int(0)] = "/home/ian/R/x86_64-pc-linux-gnu-library/3.4"

thatpackage = rpackages.importr('dplyr', lib_loc = "/home/ian/R/x86_64-pc-linux-gnu-library/3.4")
thatpackage.chooseCRANmirror(ind = int(1))

R[write to console]: Error: package or namespace load failed for ‘dplyr’:
 package ‘dplyr’ was installed by an R version with different internals; it needs to be reinstalled for use with this R version

R[write to console]: Error: package ‘dplyr’ was installed by an R version with different internals; it needs to be reinstalled for use with this R version



RRuntimeError: Error: package ‘dplyr’ was installed by an R version with different internals; it needs to be reinstalled for use with this R version


In [218]:
# Using R inside python
import rpy2
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects.packages import importr
base = importr('base')
print(base._libPaths()[0])

#d = {'print.me': 'print_dot_me', 'print_me': 'print_uscore_me'}
try:
	thatpackage = importr('fibrationSymmetries', lib_loc = "/home/ian/R/x86_64-pc-linux-gnu-library/3.4")
except:
	try:
		thatpackage = importr('fibrationSymmetries', lib_loc = "/usr/local/lib/R/site-library")
	except:
		try:
			thatpackage = importr('fibrationSymmetries', lib_loc = "/usr/lib/R/site-library")
		except:
			thatpackage = importr('fibrationSymmetries', lib_loc = "/usr/lib/R/library")

#fibrationSymmetries = rpackages.importr('dplyr')

#with localconverter(robjects.default_converter + pandas2ri.converter):
#	r_from_pd_df = robjects.conversion.py2rpy(symmetryAnalyzer.edges[['Source', 'Target']])

#fibrationSymmetries.get.balanced.coloring.Kamei(raw_edges = r_from_pd_df)

R[write to console]: Error: package or namespace load failed for ‘fibrationSymmetries’:
 package ‘fibrationSymmetries’ was installed by an R version with different internals; it needs to be reinstalled for use with this R version

R[write to console]: Error in loadNamespace(name) : 
  there is no package called ‘fibrationSymmetries’

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/local/lib/R/site-library’ contains no packages

R[write to console]: 2: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



[1] "/home/ian/sage/local/lib/R/library"



PackageNotInstalledError: The R package "fibrationSymmetries" is not installed.

In [207]:
# Using R inside python
import rpy2
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects.packages import importr
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind = int(1))

# Install packages
packnames = ('hunspell', 'some other desired packages')
utils.install_packages(StrVector(packnames))

# Load packages
hunspell = importr('hunspell')

R[write to console]: also installing the dependencies ‘Rcpp’, ‘digest’


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/Rcpp_1.0.7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2941808 bytes (2.8 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to co

PackageNotInstalledError: The R package "hunspell" is not installed.

In [200]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

r = robjects.r
r['source']('balanced_coloring.R')
get_balanced_coloring = robjects.globalenv['get_balanced_coloring']

with localconverter(robjects.default_converter + pandas2ri.converter):
	r_from_pd_df = robjects.conversion.py2rpy(symmetryAnalyzer.edges[['Source', 'Target']])
	
get_balanced_coloring(r_from_pd_df)

R[write to console]: Loading required package: fibrationSymmetries

R[write to console]: Error in get.balanced.coloring.Kamei(raw_edges = edges) : 
  could not find function "get.balanced.coloring.Kamei"

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
R[write to console]: 
 
R[write to console]:  there is no package called ‘fibrationSymmetries’



RRuntimeError: Error in get.balanced.coloring.Kamei(raw_edges = edges) : 
  could not find function "get.balanced.coloring.Kamei"


In [157]:
from scipy.spatial import distance_matrix

symmetryAnalyzer = SymmetryAnalyzer("test.txt")
#symmetryAnalyzer = SymmetryAnalyzer("../DATA/INTEGER-PROGRAM-COST-EDGES/INPUT/c_fw_no_07_06/run_c_fw_no_nc_9.txt")
symmetryAnalyzer.decompose()
symmetryAnalyzer.calculate_indices()
symmetryAnalyzer.print_indices()

graph_laplacian = np.array(symmetryAnalyzer.ig_graph.laplacian(normalized = False))

eigenval, eigenvect = np.linalg.eig(graph_laplacian)

print(np.sort(eigenval))
print(eigenvect[0])
print(eigenvect[2])
print(eigenvect[4])
print(eigenvect[1])
print(eigenvect[3])
#[print(eigenvect[i]) for i in range(len(eigenvect))]
pd.DataFrame(distance_matrix(eigenvect, eigenvect, p = 2))

Fiedler value =  1.9999999999999998
Normalized Fiedler value =  1.0
Eigen ratio =  0.3999999999999998
Mean epsilon =  5.0
Mean epsilon all =  1
[-1.16923066e-16  2.00000000e+00  2.00000000e+00  5.00000000e+00
  5.00000000e+00]
[-0.36514837  0.81649658 -0.4472136   0.06942101  0.09365858]
[-0.36514837 -0.40824829 -0.4472136   0.06942101 -0.74926865]
[-0.36514837 -0.40824829 -0.4472136   0.06942101  0.65561007]
[ 5.47722558e-01 -7.66820847e-17 -4.47213595e-01  5.90078614e-01
  1.56225498e-17]
[ 5.47722558e-01  3.68708168e-16 -4.47213595e-01 -7.98341655e-01
  1.21857053e-16]


,0,1,2,3,4
0,0.000000,1.334113,1.486784,1.503923,1.347512
1,1.334113,0.000000,1.353694,1.388420,1.304189
2,1.486784,1.353694,0.000000,1.521320,1.404879
3,1.503923,1.388420,1.521320,0.000000,1.477443
4,1.347512,1.304189,1.404879,1.477443,0.000000
